In [233]:
import os
import pandas as pd
import pydicom
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import scipy
from scipy import ndimage

from skimage import measure

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import numpy as np

from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.tools import FigureFactory as FF
from plotly.graph_objs import *

from scipy.ndimage import zoom
from scipy.stats import kurtosis
from scipy.stats import skew

from tqdm import tqdm

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_addons as tfa
from tensorflow.keras.layers.experimental import preprocessing

import keras

os.getcwd()

import warnings
warnings.filterwarnings("ignore")

In [175]:
#Extract train.csv into a dataframe
data_path='../input/osic-pulmonary-fibrosis-progression'
traindf=pd.read_csv(f'{data_path}/train.csv')

traindf.head()

,Patient,Weeks,FVC,Percent,Age,Sex,SmokingStatus
0,ID00007637202177411956430,-4,2315,58.253649,79,Male,Ex-smoker
1,ID00007637202177411956430,5,2214,55.712129,79,Male,Ex-smoker
2,ID00007637202177411956430,7,2061,51.862104,79,Male,Ex-smoker
3,ID00007637202177411956430,9,2144,53.950679,79,Male,Ex-smoker
4,ID00007637202177411956430,11,2069,52.063412,79,Male,Ex-smoker


In [176]:
coredf=pd.DataFrame({'pid': traindf['Patient'].unique()})

In [177]:
#Converting traindf to coredf

n_weeks=[]
weeks=[]
fvc=[]
percent=[]
age=[]
sex=[]
smoking=[]


for pid in coredf['pid']:
    tempdf=traindf[traindf['Patient']==pid]
    
    n_weeks.append(len(tempdf))
    weeks.append(list(tempdf['Weeks'].values))
    fvc.append(list(tempdf['FVC'].values))
    percent.append(list(tempdf['Percent'].values))
    age.append(tempdf['Age'].values[0])
    sex.append(tempdf['Sex'].values[0])
    smoking.append(tempdf['SmokingStatus'].values[0])
    

coredf['n_weeks']=n_weeks
coredf['weeks']=weeks
coredf['fvc']=fvc
coredf['percent']=percent
coredf['age']=age
coredf['sex']=sex
coredf['smoking']=smoking

display(coredf)

,pid,n_weeks,weeks,fvc,percent,age,sex,smoking
0,ID00007637202177411956430,9,"[-4, 5, 7, 9, 11, 17, 29, 41, 57]","[2315, 2214, 2061, 2144, 2069, 2101, 2000, 206...","[58.2536487166583, 55.7121288374434, 51.862103...",79,Male,Ex-smoker
1,ID00009637202177434476278,9,"[8, 9, 11, 13, 15, 22, 33, 45, 60]","[3660, 3610, 3895, 3759, 3639, 3578, 3625, 339...","[85.28287818063191, 84.11781153881999, 90.7586...",69,Male,Ex-smoker
2,ID00010637202177584971671,9,"[0, 1, 3, 5, 7, 13, 25, 37, 54]","[3523, 3373, 3327, 2993, 3030, 3103, 2993, 247...","[94.72467197246719, 90.6915465691547, 89.45472...",60,Male,Ex-smoker
3,ID00011637202177653955184,9,"[6, 7, 9, 11, 13, 19, 32, 43, 58]","[3326, 3419, 3541, 3502, 3410, 3477, 3269, 334...","[85.9875904860393, 88.3919338159255, 91.546018...",72,Male,Ex-smoker
4,ID00012637202177665765362,9,"[33, 35, 37, 39, 41, 47, 58, 71, 87]","[3418, 3759, 3276, 3443, 3268, 3449, 3324, 323...","[93.7260063617418, 103.07666995722299, 89.8321...",65,Male,Never smoked
...,...,...,...,...,...,...,...,...
171,ID00419637202311204720264,9,"[6, 7, 9, 10, 13, 18, 31, 43, 59]","[3020, 2859, 2783, 2719, 2738, 2694, 2708, 279...","[70.18685507111651, 66.4451055126894, 64.67881...",73,Male,Ex-smoker
172,ID00421637202311550012437,10,"[15, 17, 19, 21, 23, 29, 41, 54, 70, 70]","[2739, 2756, 2755, 2820, 2853, 2716, 2833, 277...","[82.0452911574407, 82.55451713395641, 82.52456...",68,Male,Ex-smoker
173,ID00422637202311677017371,8,"[6, 11, 13, 15, 17, 23, 35, 47]","[1930, 1936, 1955, 1848, 1897, 1946, 1862, 1713]","[76.6724932464643, 76.91085332909579, 77.66566...",73,Male,Ex-smoker
174,ID00423637202312137826377,9,"[17, 18, 19, 21, 23, 30, 42, 53, 70]","[3294, 2777, 2700, 3014, 2661, 2778, 2516, 243...","[79.2589027911453, 66.8190567853705, 64.966313...",72,Male,Ex-smoker


In [178]:
#Adding dicom scan details to coredf
dicoms=[]
n_dicoms=[]
bad_ids=[]

for pid in coredf['pid']:
    try:
        files=os.listdir(f"{data_path}/train/{pid}")

        for i in range(len(files)):
            files[i]=files[i][:-4]

        dicoms.append(files)
        n_dicoms.append(len(files))
    
    except:
        bad_ids.append(pid)

coredf['dicoms']=dicoms
coredf['n_dicoms']=n_dicoms

display(coredf)

,pid,n_weeks,weeks,fvc,percent,age,sex,smoking,dicoms,n_dicoms
0,ID00007637202177411956430,9,"[-4, 5, 7, 9, 11, 17, 29, 41, 57]","[2315, 2214, 2061, 2144, 2069, 2101, 2000, 206...","[58.2536487166583, 55.7121288374434, 51.862103...",79,Male,Ex-smoker,"[12, 18, 9, 22, 25, 14, 11, 24, 29, 23, 10, 28...",30
1,ID00009637202177434476278,9,"[8, 9, 11, 13, 15, 22, 33, 45, 60]","[3660, 3610, 3895, 3759, 3639, 3578, 3625, 339...","[85.28287818063191, 84.11781153881999, 90.7586...",69,Male,Ex-smoker,"[377, 257, 240, 317, 12, 323, 210, 327, 253, 1...",394
2,ID00010637202177584971671,9,"[0, 1, 3, 5, 7, 13, 25, 37, 54]","[3523, 3373, 3327, 2993, 3030, 3103, 2993, 247...","[94.72467197246719, 90.6915465691547, 89.45472...",60,Male,Ex-smoker,"[12, 18, 9, 97, 95, 22, 96, 25, 71, 39, 45, 14...",106
3,ID00011637202177653955184,9,"[6, 7, 9, 11, 13, 19, 32, 43, 58]","[3326, 3419, 3541, 3502, 3410, 3477, 3269, 334...","[85.9875904860393, 88.3919338159255, 91.546018...",72,Male,Ex-smoker,"[12, 18, 9, 22, 25, 14, 11, 24, 29, 23, 10, 28...",31
4,ID00012637202177665765362,9,"[33, 35, 37, 39, 41, 47, 58, 71, 87]","[3418, 3759, 3276, 3443, 3268, 3449, 3324, 323...","[93.7260063617418, 103.07666995722299, 89.8321...",65,Male,Never smoked,"[12, 18, 9, 22, 25, 39, 45, 14, 11, 44, 24, 34...",49
...,...,...,...,...,...,...,...,...,...,...
171,ID00419637202311204720264,9,"[6, 7, 9, 10, 13, 18, 31, 43, 59]","[3020, 2859, 2783, 2719, 2738, 2694, 2708, 279...","[70.18685507111651, 66.4451055126894, 64.67881...",73,Male,Ex-smoker,"[12, 18, 9, 22, 25, 14, 11, 24, 23, 10, 28, 17...",28
172,ID00421637202311550012437,10,"[15, 17, 19, 21, 23, 29, 41, 54, 70, 70]","[2739, 2756, 2755, 2820, 2853, 2716, 2833, 277...","[82.0452911574407, 82.55451713395641, 82.52456...",68,Male,Ex-smoker,"[12, 18, 9, 22, 25, 39, 45, 14, 11, 44, 24, 61...",62
173,ID00422637202311677017371,8,"[6, 11, 13, 15, 17, 23, 35, 47]","[1930, 1936, 1955, 1848, 1897, 1946, 1862, 1713]","[76.6724932464643, 76.91085332909579, 77.66566...",73,Male,Ex-smoker,"[377, 257, 240, 452, 317, 12, 404, 424, 323, 2...",473
174,ID00423637202312137826377,9,"[17, 18, 19, 21, 23, 30, 42, 53, 70]","[3294, 2777, 2700, 3014, 2661, 2778, 2516, 243...","[79.2589027911453, 66.8190567853705, 64.966313...",72,Male,Ex-smoker,"[257, 240, 12, 210, 253, 120, 208, 141, 18, 9,...",290


In [179]:
coredf = coredf.sort_values('n_dicoms')
coredf=coredf.reset_index()

In [180]:
def get_scan(path,index):
    dicom = pydicom.dcmread(os.path.join(path,os.listdir(path)[0]))
    
    return dicom

def get_scans(path):
    dicoms=[]

    if len(os.listdir(path))<1000:
        for filename in os.listdir(path):
            dicoms.append(pydicom.dcmread(os.path.join(path,filename)))
    else:
        return dicoms
    
    return dicoms

def get_images(scans):
    temp=[]

    if len(scans)>0:
        for i in range(len(scans)):
            try:
                image=scans[i].pixel_array
            except:
                return None

            dtype=image
            image[image<=-1000] = 0

            intercept = scans[i].RescaleIntercept
            slope = scans[i].RescaleSlope

            if slope != 1:
                image = slope * images


            temp.append(image)

    else:
        return None

    return np.asarray(temp,dtype=np.int16)

def crop_image(image):
    if image.shape != (512,512):
        left=((image.shape[0]-512)//2)
        right=image.shape[0]-left
        top=((image.shape[1]-512)//2)
        bottom=image.shape[1]-top
        
        return image[left-1:right-1,top-1:bottom-1]
    else:
        return image
    
def crop_images(images):
    if images[0].shape != (512,512):
        temp=[]
        for image in images:
            temp.append(crop_image(image))
        
        return np.asarray(temp)
    else:
        return images

def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
    spacing=[scan[0].SliceThickness,scan[0].PixelSpacing[0],scan[0].PixelSpacing[1]]
    spacing = np.array(list(spacing))

    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image


def dimension_scaling(images):
  '''
  Returns images with channels. e.g. converts 512x512 images to 512x512x3

  Inputs:
  imageslices:  NumpyArray  Imageslices extracted from the DICOM files using fnc get_images_from_slices

  '''

  images=np.expand_dims(images,axis=-1)
  images=np.concatenate((images,)*3, axis=-1)

  return np.asarray(images)

def normalize(arr): 
    return (arr-arr.min())/(arr.max()-arr.min())
    

def get_window_value(feature):
    if type(feature) == pydicom.multival.MultiValue:
        return np.int(feature[0])
    else:
        return np.int(feature)

In [181]:
#Standardize the pixel values
def make_lungmask(img, display=False):
    row_size= img.shape[0]
    col_size = img.shape[1]
    
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    # Find the average pixel value near the lungs
    # to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    # To improve threshold finding, I'm moving the 
    # underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    #
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    #
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.  
    # We don't want to accidentally clip the lung.

    eroded = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0

    #
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation

    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        
        plt.show()
        
    return mask

In [182]:
m = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b6/feature-vector/1",
                   trainable=False),  # Can be True, see below.
    tf.keras.layers.Dense(7, activation='relu')
])
m.build([None, 512, 512, 3])

In [ ]:
area=[]
pixelspacing_r = []
pixelspacing_c = []
slice_thicknesses = []
window_widths = []
window_levels = []
eff_features=[]
r_distance=[]
c_distance=[]
area_cm2=[]
slice_volume_cm3=[]

bad_ids=[]

count=0


for pid in coredf['pid'][::-1]:
    count+=1
    images=get_images(get_scans(f'{data_path}/train/{pid}'))
    
    if type(images)!= type(None):
        images=crop_images(images)
        images=normalize(images)
        
        for i in tqdm(range(len(images)),leave=True,desc=f"{count}"):
            try:
                images[i]=images[i]*make_lungmask(images[i],False)
            except:
                images[i]=images[i]*np.zeros((512,512))
        
        images=dimension_scaling(images)
        
        features=np.mean(m.predict(images),axis=0)
        
        eff_features.append(features)

    else:
        bad_ids.append(pid)
        eff_features.append(np.zeros(7,np.float32))

    scan = get_scan(f'{data_path}/train/{pid}',0)
    
    area.append(scan.Columns*scan.Rows)
    
    window_widths.append(get_window_value(scan.WindowWidth))
    window_levels.append(get_window_value(scan.WindowCenter))
    
    spacing = scan.PixelSpacing
    pixelspacing_r.append(spacing[0])
    pixelspacing_c.append(spacing[1])
    
    slice_thicknesses.append(scan.SliceThickness)
    
    r_distance.append(pixelspacing_r[-1] * scan.Rows)
    c_distance.append(pixelspacing_c[-1] * scan.Columns)
    area_cm2.append(0.1 * r_distance[-1] * 0.1 * c_distance[-1])
    slice_volume_cm3.append(0.1 * slice_thicknesses[-1] * area_cm2[-1])

In [ ]:
import pickle
lungdata=[area,pixelspacing_r,pixelspacing_c,slice_thicknesses,window_widths,window_levels,eff_features,r_distance,c_distance,area_cm2,slice_volume_cm3]

with open("lungdata.pickle","wb") as file:
  pickle.dump(lungdata,file)

In [183]:
pickledir="./lungdata.pickle"
area,pixelspacing_r,pixelspacing_c,slice_thicknesses,window_widths,window_levels,eff_features,r_distance,c_distance,area_cm2,slice_volume_cm3=pickle.load(open(pickledir,"rb"))

In [184]:
coredf['area']=area
coredf['psr']=pixelspacing_r
coredf['psc']=pixelspacing_c
coredf['st']=slice_thicknesses
coredf['ww']=window_widths
coredf['wl']=window_levels
coredf['rd']=r_distance
coredf['cd']=c_distance
coredf['areacm']=area_cm2
coredf['volcm']=slice_volume_cm3


for i in range(len(eff_features[0])):
    temp=[]
    
    for a in eff_features:
        temp.append(a[i])
    
    coredf[f"e{i}"]=temp

In [185]:
for i in range(len(coredf)):
    if coredf['sex'][i]=='Male':
        coredf['sex'][i]=0
    else:
        coredf['sex'][i]=1

    if coredf['smoking'][i] == 'Ex-smoker':
        coredf['smoking'][i]=0
    elif coredf['smoking'][i] == 'Never smoked':
        coredf['smoking'][i]=1
    else:
        coredf['smoking'][i]=2

In [200]:
stats=['age','sex','smoking','area',
        'psr','psc','st','ww','wl','rd','cd','areacm',
       'volcm','e0','e1','e2','e3','e4','e5','e6'
      ]

trainx=[]

for i in tqdm(range(len(coredf))):
    weeks=coredf['weeks'][i]
    fvc=coredf['fvc'][i]
    percent=coredf['percent'][i]
    
    for x in range(len(weeks)):        
        for y in range(x+1,len(weeks)):
            temp=[]
            
            temp.append(weeks[x])    #baseweek
            temp.append(percent[x])  #basepercent
            temp.append(fvc[x])      #basefvc
            
            temp.append(weeks[y])    #current week
            temp.append(weeks[y]-weeks[x]) #change in weeks
            
            for z in range(len(stats)):
                temp.append(coredf[stats[z]][i])
            
            temp.append(fvc[y])    #current fvc
            
            trainx.append(temp)

100%|██████████| 176/176 [00:01<00:00, 158.40it/s]


In [202]:
cols=['baseweek','basepercent','basefvc','current_week','diff(weeks)','current_fvc']

for feature in stats:
    cols.append(feature)
    
modeldf = pd.DataFrame(trainx, columns = cols)

In [204]:
for col in modeldf.columns:
    modeldf[col]=normalize(modeldf[col].values)

In [206]:
px.imshow(modeldf.corr())

In [215]:
initializer = tf.keras.initializers.GlorotNormal()

In [208]:
def laplace_log_likelihood(actual_fvc, predicted_fvc, confidence, return_values = False):
    """
    Calculates the modified Laplace Log Likelihood score for this competition.
    """
    sd_clipped = np.maximum(confidence, 70)
    delta = np.minimum(np.abs(actual_fvc - predicted_fvc), 1000)
    metric = np.sqrt(2) * delta / sd_clipped + np.log(np.sqrt(2) * sd_clipped)

    if return_values:
        return metric
    else:
        return np.mean(metric)

In [234]:
C1, C2 = tf.constant(70, dtype='float32'), tf.constant(1000, dtype="float32")
#=============================#
def score(y_true, y_pred):
    tf.dtypes.cast(y_true, tf.float32)
    tf.dtypes.cast(y_pred, tf.float32)
    sigma = y_pred[:, 2] - y_pred[:, 0]
    fvc_pred = y_pred[:, 1]
    
    #sigma_clip = sigma + C1
    sigma_clip = tf.maximum(sigma, C1)
    delta = tf.abs(y_true[:, 0] - fvc_pred)
    delta = tf.minimum(delta, C2)
    sq2 = tf.sqrt( tf.dtypes.cast(2, dtype=tf.float32) )
    metric = ((delta / sigma_clip)*sq2 + tf.math.log(sigma_clip* sq2))
    
    return keras.backend.mean(metric)
#============================#

def qloss(y_true, y_pred):
    # Pinball loss for multiple quantiles
    qs = [0.2, 0.50, 0.8]
    q = tf.constant(np.array([qs]), dtype=tf.float32)
    e = y_true - y_pred
    v = tf.maximum(q*e, (q-1)*e)
    return keras.backend.mean(v)
#=============================#
def mloss(_lambda):
    def loss(y_true, y_pred):
        return _lambda * qloss(y_true, y_pred) + (1 - _lambda)*score(y_true, y_pred)
    return loss
#=================
def make_model():
    ini_input=keras.Input(shape=(25,1))

    y = keras.layers.LSTM(100,return_sequences=True)(ini_input)
    y = keras.layers.LSTM(200,return_sequences=True)(y)
    y = keras.layers.LSTM(100,return_sequences=True)(y)
    y = keras.layers.Reshape([100*25])(y)
    x = keras.layers.Dense(100,kernel_initializer=initializer, activation="relu", name="d1")(y)
    x = keras.layers.Dense(100,kernel_initializer=initializer, activation="relu", name="d2")(x)
    p1 = keras.layers.Dense(3, activation="linear", name="p1")(x)
    p2 = keras.layers.Dense(3, activation="relu", name="p2")(x)
    preds = keras.layers.Lambda(lambda x: x[0] + tf.cumsum(x[1], axis=1), 
                     name="preds")([p1, p2])

    model=keras.Model(inputs=ini_input,outputs=preds)

    model.compile(loss=mloss(0.8),optimizer=tf.keras.optimizers.Adam(lr=0.2, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.01, amsgrad=False),metrics=[score])
    
    return model

In [235]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5,shuffle=True)

trainx = modeldf[modeldf.columns[:-1]].values
trainy = modeldf[modeldf.columns[-1]].values

In [232]:
b_size=128  
count=1
history={}

for train_ind,val_ind in kf.split(trainx):
    print(f'Fold No. {count}')
    
    model=make_model()
    
    temp={}
    
    print('Training....')
    
    temp['train'] = model.fit(trainx[train_ind],trainy[train_ind],batch_size=b_size,epochs=100, verbose=0)
    
    print(f'Training score to beat: {laplace_log_likelihood(trainy[train_ind],np.mean(trainy[train_ind]),np.std(trainy[train_ind]))}')
    
    print(model.evaluate(trainx[train_ind],trainy[train_ind],batch_size=b_size))
    
    print(f'Validation score to beat: {laplace_log_likelihood(trainy[val_ind],np.mean(trainy[val_ind]),np.std(trainy[val_ind]))}')
    temp['test'] = model.evaluate(trainx[val_ind],trainy[val_ind],batch_size=b_size,verbose=1)
    
    history[count]=temp
    
    count+=1
    

Fold No. 1
Training....


AttributeError: in user code:

    /opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    <ipython-input-231-3da7c25e7946>:33 accuracy  *
        y_true=y_true.numpy()

    AttributeError: 'Tensor' object has no attribute 'numpy'


In [ ]:
x=model.predict(trainx[val_ind])
x=x.mean(axis=1)

In [ ]:
plt.plot(x)
plt.plot(trainy[val_ind])

In [ ]:
import sklearn
sklearn.metrics.r2_score(trainy[val_ind], x)

In [ ]:
model.summary()

In [ ]:
trainy[train_ind][0]

In [ ]:
y_pred= np.array([[1990,2000,2010],
                  [1980,2002,2010]])


print(y_pred[1] - y_pred[0])

In [ ]:
model.summary()